# Mentoría Ciencia de datos aplicada a la predicción de licencias médicas y comportamiento de los colaboradores

# Trabajo Práctico 2: Análisis Exploratorio y Curación

Trabajar con el csv denominado data.csv y resolver las siguientes consignas:

1) Analisis e imputación de valores faltantes, nulos y/o ruidosos en las columnas relevantes del dataset. 

    1.A) ¿Existe alguna relación entre los datos faltantes? Usar la librería missingno para resolver esta consigna.
    1.B) Realizar la imputación de aquellas columnas con datos faltantes, justificar el método.
    1.C) Trabajar con las columnas dias_aprobados y dias_solicitados, observar si existe coherencia en las mismas y datos nulos o ruidosos que deban imputar. 

2) Observar si existen outliers en las columnas relevantes. Tomar una decisión con los mismos y justificarla. 

3) Trabajar con las columnas categóricas del dataset, decidir si corresponde realizar algún reagrupamiento, considerando su frecuencia. 
   Ayuda: tener en cuenta la presencia de caracteres especiales y/o espacios vacíos en la columna diagnostico.
   
   3.1) Para el reagrupamiento de la columna diagnostico, ayudarse de alguna base externa, como por ejemplo https://icdcode.info/espanol/cie-10/codigos.html que contiene la clasificación y agrupación de diagnosticos a nivel internacional. Les dejamos el csv de esta base https://raw.githubusercontent.com/verasativa/CIE-10/master/cie-10.csv. 
   
4) A partir de la columna fecha_creacion, generar dos columnas extras temporales utilizando las escalas temporales que consideren adecuadas, por ejemplo: día, semana, quincena, mes, trimestre o semestre. Pueden utilizar la función resample de pandas. 

5) Generar y guardar el nuevo dataset con todas las tranformaciones realizadas, el cual sera el input de los siguientes tp de aprendizaje supervisado/no supervisado. 

## Carga del dataset

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

url = 'https://raw.githubusercontent.com/MentoriaDiploDatos/Presentacion/main/data.csv'
df = pd.read_csv(url, sep=';')

In [2]:
df.head(2)

,numero_licencia,fecha_creacion,dispositivo_acceso,dias_solicitados,fecha_inicio,fecha_fin,diagnostico,id_diagnostico,id_usuario,fecha_nacimiento,genero,tipo_licencia,estado_civil,dias_aprobados,decision_medica,id_auditor,motivo_rechazo,categoria_diagnostico
0,1,2021-01-01 00:10:10.418,APP,2.0,2021-01-01,2021-01-01,EFECTOS ADVERSOS DE VACUNAS VIRALES,NaN,15508,1979-11-28,FEMALE,Médica,OTHER,2.0,EDITAR_Y_APROBAR,14721.0,NaN,NaN
1,105,2021-01-02 14:14:20.330,APP,2.0,2021-01-02,2021-01-03,EFECTOS ADVERSOS DE VACUNAS VIRALES,NaN,15508,1979-11-08,FEMALE,Médica,OTHER,2.0,EDITAR_Y_APROBAR,14721.0,NaN,NaN
